# 文本预处理

一篇文章可以看作字符或单词的序列，常见文本数据的预处理一般包括4个步骤：
1. 读入文本
2. 分词
3. 建立字典，将单词映射成索引
4. 将文本从词的序列转为索引的序列

以下以英文小说[Time Machine](http://www.gutenberg.org/ebooks/35)为例，英文。

In [1]:
import collections
import re
import nltk

## 读入文本

In [2]:
def read_time_machine():
    with open("timemachine.txt","r",encoding="utf-8") as f:
        # 仅保留a-zA-Z的字符
        lines = [re.sub('[^a-z]+',' ',line.strip().lower()) for line in f]
    return lines

In [3]:
lines = read_time_machine()
len(lines)

3702

## 分词
字-**词**-句-**文章**

**第一层列表并不是句子，因为输入并不是一个句子一行。**

In [4]:
def tokenize(sentences, token="word"):
    if token == "word":
        return [sentence.split(" ") for sentence in sentences 
                                        if sentence.strip() != "" and 
                                           len(sentence.split(" "))>1]
    elif token == "char":
        return [list(sentence) for sentence in sentences]
    else:
        print("Error：unknown token type"+token)        

In [5]:
tokens = tokenize(lines)
tokens[0]

['the', 'time', 'machine', 'by', 'h', 'erbert', 'g', 'eorge', 'wells', '']

## 建立字典

In [6]:
def count_corpus(sentences):
    tokens = [tk for st in sentences for tk in st]
    return collections.Counter(tokens)  # 返回一个字典，记录每个词的出现次数

In [7]:
len(count_corpus(tokens))

4932

In [8]:
class Vocab():
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        counter = count_corpus(tokens)
        self.token_freqs = list(counter.items())
        self.idx_to_token = []
        if use_special_tokens:
            # padding, begin of sentence, end of sentence, unknown
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            self.idx_to_token += ['', '', '', '']
        else:
            self.unk = 0
            self.idx_to_token += ['']
        self.idx_to_token += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in self.idx_to_token]
        self.token_to_idx = dict()
        for idx, token in enumerate(self.idx_to_token):
            self.token_to_idx[token] = idx
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

In [9]:
vocab = Vocab(tokens)

In [10]:
print(list(vocab.token_to_idx.items())[0:10])

[('', 0), ('the', 1), ('time', 2), ('machine', 3), ('by', 4), ('h', 5), ('erbert', 6), ('g', 7), ('eorge', 8), ('wells', 9)]


In [11]:
vocab[["the","time"]]

[1, 2]

In [12]:
vocab.to_tokens([2,3])

['time', 'machine']

## 将词转为索引

In [31]:
for i in range(0, 10):
    print("line {} :".format(i),len(tokens[i]))
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

line 0 : 10
words: ['the', 'time', 'machine', 'by', 'h', 'erbert', 'g', 'eorge', 'wells', '']
indices: [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
line 1 : 12
words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of']
indices: [1, 2, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
line 2 : 11
words: ['him', 'was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes']
indices: [20, 21, 22, 23, 24, 25, 17, 26, 27, 28, 29]
line 3 : 11
words: ['shone', 'and', 'twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and']
indices: [30, 31, 32, 31, 27, 33, 34, 35, 21, 36, 31]
line 4 : 11
words: ['animated', 'the', 'fire', 'burned', 'brightly', 'and', 'the', 'soft', 'radiance', 'of', 'the']
indices: [37, 1, 38, 39, 40, 31, 1, 41, 42, 19, 1]
line 5 : 10
words: ['incandescent', 'lights', 'in', 'the', 'lilies', 'of', 'silver', 'caught', 'the', 'bubbles']
indices: [43, 44, 45, 1, 46, 19, 47, 48, 1, 49]
line 6 : 11
words: ['that', 'flas

## 用现有工具进行分词
增加的功能
- 标点符号的语义信息
- doesn't等缩略词的处理

In [14]:
text = "Mr. Chen doesn't agree with my suggestion."

### spacy

In [15]:
import spacy
# python -m spacy download en_core_web_sm

In [19]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

In [17]:
print([token.text for token in doc])

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']


### nltk

In [28]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import data

In [29]:
print(word_tokenize(text))

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']


# 语言模型

一段自然语言文本可以看作是一个离散时间序列，给定一个长度为T的词的序列，语言模型的目标是评估该序列是否合理，即计算该序列的概率。

- 语料库：corpus

语言模型的参数是**词的概率**以及**给定前n个词的情况下的条件概率**

## n元语法（n-gram）

n元语法通过马尔可夫假设简化模型，马尔可夫假设是指一个词的出现只与前面(n-1个相关，即n阶马尔科夫链。

语言模型的n阶马尔可夫链，即n-gram。

$$
P(w_1, w_2, \ldots, w_T) \approx \prod_{t=1}^T P(w_t \mid w_{t-(n-1)}, \ldots, w_{t-1}) .
$$

【常用的n-gram模型】
- unigram：(2)
- bigram：(3)
- trigram

$$\begin{split}\begin{aligned}
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2) P(w_3) P(w_4) ,\\
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_2) P(w_4 \mid w_3) ,\\
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_1, w_2) P(w_4 \mid w_2, w_3) .
\end{aligned}\end{split}$$

【n-gram缺陷】
1. 参数空间过大
2. 数据过于稀疏(齐夫定律）

## 周杰伦歌词数据集
从[d2lzh的github](https://github.com/d2l-ai/d2l-zh/blob/master/data/jaychou_lyrics.txt.zip)上下载。

In [35]:
import zipfile
with zipfile.ZipFile('jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')

In [38]:
print(len(corpus_chars))
print(corpus_chars[: 40])
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[: 10000]

10000
想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每


### 建立字符索引

In [40]:
idx_to_char = list(set(corpus_chars))
char_to_idx = {char:i for i,char in enumerate(idx_to_char)}
vocab_size = len(char_to_idx)

In [42]:
vocab_size

1027

In [45]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]  # 将每个字符转化为索引，得到一个索引的序列
sample = corpus_indices[: 20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [833, 272, 249, 217, 146, 124, 763, 833, 272, 2, 901, 573, 897, 620, 493, 693, 763, 833, 272, 2]


In [47]:
def load_data_jay_lyrics():
    with zipfile.ZipFile('jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

## 时序数据的采样
时间序列有大量重复的样本，因此需要采样。

序列长度为T，时间步数为n，共有$T-n$个合法样本。

- 随机采样
- 相邻采样

### 随机采样

下面的代码每次从数据里随机采样一个小批量。其中批量大小`batch_size`是每个小批量的样本数，`num_steps`是每个样本所包含的时间步数。
在随机采样中，每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻。

**硬性分成了多个片段，片段之间的联系破坏了。**

In [55]:
import random
import torch

In [56]:
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    num_examples = (len(corpus_indices)-1)//num_steps
    # example_indices记录每个分组的第一个下标
    example_indices = [i*num_steps for i in range(num_examples)]
    random.shuffle(example_indices)
    
    def _data(i):
        return corpus_indices[i:i+num_steps]
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )
    for i in range(0,num_examples, batch_size):
        batch_indices = example_indices[i:i+batch_size]
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)
        

In [57]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]], device='cuda:0') 
Y: tensor([[13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24]], device='cuda:0') 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [ 0,  1,  2,  3,  4,  5]], device='cuda:0') 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [ 1,  2,  3,  4,  5,  6]], device='cuda:0') 



### 相邻采样

在相邻采样中，相邻的两个随机小批量在原始序列上的位置相毗邻。

也把corpus序列分成了多个片段。

In [58]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [61]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]], device='cuda:0') 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]], device='cuda:0') 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]], device='cuda:0') 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]], device='cuda:0') 



# 循环神经网络基础